In [1]:
import glob
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.features.feature_metadata import FeatureMetadata

In [2]:
training_files = 5
data_files = glob.glob('bid_data/*.tsv')
dfs = [pd.read_csv(file, sep='\t') for file in data_files[:training_files]]

train_data = TabularDataset(pd.concat(dfs, ignore_index=True))
train_data['bid_pos'] = train_data['bid_pos'].astype('category')

In [3]:
train_data.head()

,bid_pos,go_alone,def_alone,def_pos_rel,turn_card_level,bid_turn_suit,bid_next_suit,bid_green_suit,top_trump_strg,top_2_trump_strg,top_3_trump_strg,num_trump,num_next,num_voids,num_singletons,num_off_aces,num_tricks,points
0,0,0,0,0,6,1,0,0,7,7,7,2,0,1,1,0,2,-2
1,1,1,0,0,6,1,0,0,0,0,0,0,1,1,1,2,3,1
2,2,1,0,0,6,1,0,0,0,0,0,0,2,1,1,1,0,-2
3,1,1,1,1,6,1,0,0,0,0,0,0,2,1,1,1,3,4
4,2,1,1,1,6,1,0,0,10,13,13,3,1,1,2,0,4,4


In [4]:
label = 'points'
train_data[label].describe()

count    759360.000000
mean          0.143026
std           2.625323
min          -4.000000
25%          -2.000000
50%          -1.000000
75%           4.000000
max           4.000000
Name: points, dtype: float64

In [5]:
exclude = []
outputs = ['num_tricks', 'points']
outputs.remove(label)

quantile_levels = [0.25, 0.5, 0.75]
predictor = TabularPredictor(problem_type='quantile', quantile_levels=quantile_levels, label=label)
predictor.fit(train_data.drop(columns=(exclude + outputs)), presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20240906_030638"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.9
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #202405300957~1722440358~24.04~aa0a43f~dev-Ubuntu SMP PREEMPT_DY
CPU Count:          8
Memory Avail:       12.11 GB / 15.34 GB (79.0%)
Disk Space Avail:   118.36 GB / 452.95 GB (26.1%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of Auto

In [6]:
test_data = TabularDataset(data_files[training_files])

y_pred = predictor.predict(test_data.drop(columns=([label] + exclude + outputs)))
y_pred.head()

Loaded data from: bid_data/bid_model_2-20240903_113350.tsv | Columns = 18 / 18 | Rows = 151660 -> 151660


,0.25,0.50,0.75
0,-2.000000,0.541914,1.444050
1,-2.000000,-1.851982,-1.111899
2,-1.148017,-1.000000,4.000000
3,-2.000000,-1.851982,-1.111899
4,-1.148018,-0.999999,4.000000


In [7]:
predictor.evaluate(test_data, silent=True)

{'pinball_loss': -0.6171848419095829}

In [8]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.617185,-0.619879,pinball_loss,0.336355,0.391642,123.150398,0.008142,0.078810,6.500343,2,True,4
1,CatBoost_BAG_L1,-0.617185,-0.619879,pinball_loss,0.328214,0.312832,116.650055,0.328214,0.312832,116.650055,1,True,3
2,WeightedEnsemble_L3,-0.617185,-0.619879,pinball_loss,0.332401,0.392700,126.992931,0.004188,0.079868,10.342876,3,True,8
3,LightGBMXT_BAG_L2,-0.686385,-0.689404,pinball_loss,342.277906,3700.603776,1775.831635,81.778927,965.489505,463.937981,2,True,5
4,LightGBM_BAG_L2,-0.690540,-0.692683,pinball_loss,274.961777,2805.125416,1457.437332,14.462798,70.011145,145.543679,2,True,6
5,LightGBMXT_BAG_L1,-0.699616,-0.700845,pinball_loss,233.674924,2483.501621,935.488322,233.674924,2483.501621,935.488322,1,True,1
6,LightGBM_BAG_L1,-0.732423,-0.733862,pinball_loss,26.495842,251.299818,259.755276,26.495842,251.299818,259.755276,1,True,2
7,CatBoost_BAG_L2,-0.762271,-0.762524,pinball_loss,260.826291,2735.426290,1346.232723,0.327312,0.312019,34.339070,2,True,7
